# Tarea 2

## Autor: Leonard David Vivas Dallos

## Indicaciones

1. Construya la arquitectura de una red neuronal Autoencoder en PyTorch.
2. Escriba el training loop teniendo en cuenta la técnica de Gradient Accumulation.

Para dicha parte, se puede usar el mismo dataset para ambos ejercicios.

3. El primer extra es adaptar esos puntos para funcionar con imágenes. Para esto investigar Vision Transformers, explicar la diferencias con redes convolucionales e implementar (puede usar las capas pre-existentes de PyTorch).
4. El segundo extra es escoger un dataset tabular, y usar las técnicas aprendidas de clustering (k-means y DB-Scan), investigar y hacer una implementación desde cero de BFR, y comparar el resultado de los tres métodos. ¿Qué métrica usaría?